# PDM Big Data course - final exam

## 0. Setup
The following code downloads and prepares the dataset, and puts it into the `data` folder. It is ready to execute. You will need it for the practical part (2.).

 Since it takes a couple of minutes to finish, run it now, and start answering the preliminary questions.

In [1]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 72kB/s 
     |████████████████████████████████| 204kB 39.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=7c231cdddf1af887ebe75c2b966e0e47f04b4775004775b97f0912e486d9655f
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [3]:
!wget https://github.com/CSSEGISandData/COVID-19/archive/master.zip

--2021-01-22 16:25:53--  https://github.com/CSSEGISandData/COVID-19/archive/master.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/CSSEGISandData/COVID-19/zip/master [following]
--2021-01-22 16:25:53--  https://codeload.github.com/CSSEGISandData/COVID-19/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209500304 (200M) [application/zip]
Saving to: ‘master.zip’

master.zip          100%[===================>] 199.79M   120MB/s    in 1.7s    

2021-01-22 16:25:55 (120 MB/s) - ‘master.zip’ saved [209500304/209500304]



In [4]:
!unzip -o master.zip

Archive:  master.zip
8480efad0d62f68f9a4c3c9be0952d1b474a1ad0
   creating: COVID-19-master/
 extracting: COVID-19-master/.gitignore  
  inflating: COVID-19-master/README.md  
   creating: COVID-19-master/archived_data/
  inflating: COVID-19-master/archived_data/README.md  
   creating: COVID-19-master/archived_data/archived_daily_case_updates/
 extracting: COVID-19-master/archived_data/archived_daily_case_updates/.gitignore  
  inflating: COVID-19-master/archived_data/archived_daily_case_updates/01-21-2020_2200.csv  
  inflating: COVID-19-master/archived_data/archived_daily_case_updates/01-22-2020_1200.csv  
  inflating: COVID-19-master/archived_data/archived_daily_case_updates/01-23-2020_1200.csv  
  inflating: COVID-19-master/archived_data/archived_daily_case_updates/01-24-2020_0000.csv  
  inflating: COVID-19-master/archived_data/archived_daily_case_updates/01-24-2020_1200.csv  
  inflating: COVID-19-master/archived_data/archived_daily_case_updates/01-25-2020_0000.csv  
  inflating:

In [5]:
!rm -r data
!mkdir data

rm: cannot remove 'data': No such file or directory


In [6]:
import shutil
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm

DATA_IN = Path("COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports")
DATA_OUT = Path("data")

for file_path in tqdm(list(DATA_IN.iterdir()), desc="Preprocessing"):
    if file_path.suffix != '.csv':
        shutil.copy(file_path, Path("data") / file_path.name)
        continue
    df = pd.read_csv(file_path)
    month, day, year = file_path.stem.split('.')[0].split('-')
    date = f'{year}-{month}-{day}'
    if date > '2021-01-20':
        continue
    df.rename(columns={'Lat': 'Lat_',
                       'Province/State': 'Province_State',
                       'Country/Region': 'Country_Region',
                       'Last Update': 'Last_Update'}, inplace=True)
    df = df.replace(',', '', regex=True).replace('"', '', regex=True)
    df = df[['Province_State', 'Country_Region', 'Last_Update',
             'Confirmed', 'Deaths', 'Recovered']].fillna(0)
   
    df['Date'] = date
    df.to_csv(Path("data") / file_path.name, index=False)

## 1. Preliminary questions

---
**Note**

In this part, you are expected to write Spark code. Among all Spark methods, you may only use RDD methods (`filter`, `map`, `reduceByKey`, etc.).

However, if you come up with another Spark code, that uses any other method, but still produces the expected results, you will still get half of the points.

---

### 1.1. About _replication_ and _sharding_:
* What problem(s) does each of them try to solve?
* Give one drawback of each.

Sharding is useful when the data cannot fit on a single machine (it is split across multiple machines.

Replicated storage can mitigate data loss and improve availability (if one _replica_ dies, the other still have the data and can handle requests), and also reduce response time under high load (if one replica is busy, another one can take the query).

Main drawbacks of replication: slower writes (if strong consistency) or potentially inconsistent data across replicas (if eventual consistency) ; more storage space used.

Main drawbacks of sharding: inefficient joins (if the join key is different from the sharding key) ; the master node becomes a single point of failure.

### 1.2. Among the PySpark methods `filter`, `map`, `count` and `take`, which ones are _lazy_ and which ones are _actions_? Which ones transfer data onto the master, and which ones do not?

* `filter` and `map` are lazy: they do not transfer any data onto the master (they do not do anything actually, until one executes an action on top of them).
* `count` and `take` are actions: they do transfer data onto the master.

### 1.3. Here are three separate Spark codes (wheere `sc` is a Spark context). For each of them, tell how many times the `numbers.txt` file will actually be read from the disk? Please explain each answer in a few words.
(a)
```
sc.textFile("numbers.txt")
```
(b)
```
sc.textFile("numbers.txt").cache().max()
sc.textFile("numbers.txt").cache().count()
```
(c)
```
rdd = sc.textFile("numbers.txt")
rdd.max()
rdd.count()
```

* (a) The file will be read once.
* (b) The file will be read twice, since we recreate a new RDD with the second `textFile` call. The following code would do effective caching, reading the data only once (the second `cache()` is useless):
```
rdd = sc.textFile("numbers.txt")
rdd.cache().max()
rdd.cache().count()
```
* (c) The file will be read twice (once for `max` and once for `count`), since the RDD is lazy: `textFile` does not actually store anything in memory.

## 2. Practical example

### 2.1. Read all CSV files from `data` folder into a single Spark RDD, and count the total number of rows.

In [7]:
covid_rdd_full = sc.textFile("data/*.csv").map(lambda row: row.split(","))

In [8]:
header = covid_rdd_full.first()
covid_rdd = covid_rdd_full.filter(lambda row: row[0] != "Province_State").cache()

In [9]:
covid_rdd.count()

1150435

In [10]:
index_date = header.index('Date')
index_country = header.index('Country_Region')
index_deaths = header.index('Deaths')
index_cases = header.index('Confirmed')

### 2.2. Find the last (most recent) date in the data set.


In [11]:
covid_rdd.map(lambda row: row[index_date]).reduce(lambda a, b: max(a, b))

'2021-01-20'

### 2.3. Compute the number of deaths per day worldwide.

In [12]:
daily_deaths = covid_rdd.map(lambda row: (row[index_date], float(row[index_deaths]))).reduceByKey(lambda a, b: a + b).collect()

### 2.4. Compute the mortality rate per country (over the whole time period).
The mortality rate is defined as the number of deaths divided by the number of confirmed cases.

NB: The expected answer only reads the CSV files once. But if you come up with a two-pass solution, you will still get half of the points.

In [13]:
country_mortality = covid_rdd.map(
    lambda row: (row[index_country], (float(row[index_cases]), float(row[index_deaths])))
).reduceByKey(
    lambda a, b: (a[0] + b[0], a[1] + b[1])
).map(
    lambda t: (t[0], t[1][1] / t[1][0])
).collect()

### 3. (bonus points) Same as questions 2.1, 2.2, 2.3 and 2.3 but using DataFrames instead of RDD.

### 3.1. Load the CSV files into a DataFrame and count the number of rows.

In [14]:
df = spark.read.csv("data/*.csv", header=True).cache()

In [15]:
df.count()

1150435

### 3.2. Most recent date.

In [16]:
df.agg({"Date": "max"}).show()

+----------+
| max(Date)|
+----------+
|2021-01-20|
+----------+



### 3.3. Daily number of deaths.

In [17]:
df = df.withColumn("Deaths", df["Deaths"].cast("float"))

In [18]:
df.groupby('Date').sum('Deaths').toPandas()

,Date,sum(Deaths)
0,2020-04-13,119568.0
1,2020-02-26,2770.0
2,2020-06-24,482845.0
3,2020-11-12,1295182.0
4,2020-09-12,920995.0
...,...,...
360,2020-04-05,69486.0
361,2020-12-28,1774686.0
362,2020-10-25,1154956.0
363,2020-05-01,238691.0


### 3.4. Mortality rate per country.

In [19]:
df = df.withColumn("Confirmed", df["Confirmed"].cast("float"))

In [20]:
result = df.groupby("Country_Region").agg({"Confirmed": "sum", "Deaths": "sum"})

In [21]:
result = result.withColumn('MortalityRate', result['sum(Deaths)'] / result['sum(Confirmed)'])

In [22]:
result.toPandas()

,Country_Region,sum(Deaths),sum(Confirmed),MortalityRate
0,Chad,21794.0,329300.0,0.066183
1,Paraguay,218031.0,10513560.0,0.020738
2,Russia,6179623.0,361663086.0,0.017087
3,Bahamas The,0.0,10.0,0.000000
4,North Ireland,0.0,1.0,0.000000
...,...,...,...,...
236,Vietnam,5608.0,249630.0,0.022465
237,Mainland China,77471.0,2473987.0,0.031314
238,Diamond Princess,3876.0,215024.0,0.018026
239,Mali,37189.0,891775.0,0.041702
